## 📚 Quick Reference Card

Copy-paste ready configurations for common use cases.

In [30]:
#!/usr/bin/env python3
from maya4 import get_sar_dataloader, SARTransform, SampleFilter
import functools
from maya4 import minmax_normalize, RC_MIN, RC_MAX, GT_MIN, GT_MAX
import os

DATA_DIR = os.path.abspath(os.path.join(os.getcwd(), '..', 'data'))

# ============================================================================
# 📚 COMPREHENSIVE DATALOADER CONFIGURATION REFERENCE
# ============================================================================
# Normalisation transforms for different SAR processing levels
transforms = SARTransform(
    # transform_raw: Applied to Level 0 (raw) data
    # Raw data typically has different statistics than compressed data
    transform_raw=functools.partial(
        minmax_normalize, 
        array_min=RC_MIN,  # Minimum value for normalization
        array_max=RC_MAX   # Maximum value for normalization
    ),
    
    # transform_rc: Applied to Range Compressed data
    # RC is the first compression step (range direction only)
    transform_rc=functools.partial(
        minmax_normalize,
        array_min=RC_MIN,
        array_max=RC_MAX
    ),
    
    # transform_rcmc: Applied to Range Cell Migration Corrected data
    # RCMC corrects for range cell migration due to platform motion
    transform_rcmc=functools.partial(
        minmax_normalize,
        array_min=RC_MIN,
        array_max=RC_MAX
    ),
    
    # transform_az: Applied to Azimuth focused (final) data
    # This is the ground truth - fully focused SAR image
    transform_az=functools.partial(
        minmax_normalize,
        array_min=GT_MIN,  # Different min/max for focused data
        array_max=GT_MAX
    )
)

# SampleFilter allows you to select specific SAR products based on metadata
filters = SampleFilter(
    # years: List of acquisition years to include (e.g., [2023, 2024])
    # Only products from these years will be loaded
    years=[2023, 2024],
    
    # polarizations: Radar polarization modes to include
    # Options: "hh", "hv", "vh", "vv" (H=horizontal, V=vertical)
    # "hh" = horizontal transmit, horizontal receive (co-pol)
    # "hv" = horizontal transmit, vertical receive (cross-pol)
    polarizations=["hh"],
    
    # stripmap_modes: Sentinel-1 stripmap beam modes (1-6)
    # Different modes have different swath widths and resolutions
    # Modes 1-3: narrow swaths, higher resolution
    # Modes 4-6: wider swaths, lower resolution
    stripmap_modes=[1, 2, 3],
    
    # parts: Geographic regions/partitions in the dataset
    # Maya4 organizes data into parts (PT1, PT2, etc.) by location
    # Leave empty or None to include all parts
    parts=["PT1"]
)

loader = get_sar_dataloader(
    # --- Data Sources ---
    data_dir=DATA_DIR,              # Directory containing Zarr files
    online=True,                    # True: enables remote HuggingFace access | False: local only
    
    # --- Processing Levels ---
    level_from="rcmc",              # Input SAR level: "raw", "rcmc", "az"
    level_to="az",                  # Target SAR level: "rcmc", "az"
    
    # --- Patch Configuration ---
    return_whole_image=False,       # True: return entire image as single patch | False: extract patches
    patch_mode="rectangular",       # "rectangular" (default) or "parabolic"
    patch_size=(1000, 1000),        # (height, width) | -1: computed from image minus buffer | e.g., (1, -1) for slices
    stride=(1000, 1000),            # (height, width) step size | smaller = overlapping patches
    buffer=(500, 500),              # (top/bottom, left/right) margin to avoid edge sampling
    parabola_a=0.001,               # Curvature parameter for parabolic patch mode
    
    # --- Sampling Strategy ---
    patch_order="chunk",            # "chunk": I/O efficient | "row": top-to-bottom | "col": left-to-right
    block_pattern=None,             # (n_blocks_y, n_blocks_x) for stratified sampling | None: sequential
    use_balanced_sampling=False,    # True: balance samples across products | False: natural distribution
    samples_per_prod=-1,            # Number of patches per product | -1: all available
    max_products=-1,                # Maximum number of Zarr products to use | -1: all available
    save_samples=True,              # True: save computed patch indices to disk | False: compute on-the-fly
    
    # --- Concatenation (for sequence models) ---
    concatenate_patches=False,      # True: stack patches into sequences | False: independent patches
    concat_axis=0,                  # 0: vertical stacking | 1: horizontal stacking
    max_base_sample_size=(-1, -1),  # (height, width) limit for concatenated samples | (-1, -1): unlimited
    
    # --- Data Representation ---
    complex_valued=True,            # True: native complex tensors | False: stack real/imag as channels
    positional_encoding=False,      # True: add (y, x) coordinate channels | False: data only
    
    # --- Data Filtering ---
    filters=filters,                   # SampleFilter(years=[2023, 2024], polarizations=["hh"], parts=["PT1"]) | None: no filter
    
    # --- Transformations ---
    transform=transforms,                 # SARTransform(transform_raw=..., transform_rcmc=..., transform_az=...) | callable | None
    
    # --- Performance ---
    batch_size=32,                   # Samples per batch
    num_workers=0,                  # 0: single process (debugging) | >0: parallel data loading workers
    cache_size=10,                  # Maximum number of chunks to cache in memory (LRU cache)
    backend='zarr',                 # Backend for loading: "zarr" or "dask"
    
    
    # --- Debug ---
    verbose=False,                  # True: show detailed progress | False: quiet mode
)

# --- Example Usage ---
print('✓ Dataloader configured')
for i, (x_batch, y_batch) in enumerate(loader):
    print(f'Batch {i}: X shape: {x_batch.shape} | Y shape: {y_batch.shape}')
    if i >= 2:  # Show only first 3 batches
        break

INFO:httpx:HTTP Request: GET https://huggingface.co/api/datasets?author=Maya4 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://huggingface.co/api/datasets/Maya4/PT1/tree/main?recursive=false&expand=false "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://huggingface.co/api/datasets/Maya4/PT2/tree/main?recursive=false&expand=false "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://huggingface.co/api/datasets/Maya4/PT4/tree/main?recursive=false&expand=false "HTTP/1.1 200 OK"


Total files found in remote repository: 963
✓ Dataloader configured


INFO:httpx:HTTP Request: GET https://huggingface.co/api/datasets/Maya4/PT1/tree/main/s1a-s2-raw-s-hh-20241221t213533-20241221t213558-057096-0704c4.zarr?recursive=false&expand=false "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://huggingface.co/api/datasets/Maya4/PT1/tree/main/s1a-s2-raw-s-hh-20241221t213533-20241221t213558-057096-0704c4.zarr%2Frcmc?recursive=false&expand=false "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD https://huggingface.co/datasets/Maya4/PT1/resolve/main/s1a-s2-raw-s-hh-20241221t213533-20241221t213558-057096-0704c4.zarr/rcmc/c/0/1 "HTTP/1.1 302 Found"


Trying to download file s1a-s2-raw-s-hh-20241221t213533-20241221t213558-057096-0704c4.zarr/rcmc/c/0/1 from repo Maya4/PT1


s1a-s2-raw-s-hh-20241221t213533-20241221(…):   0%|          | 0.00/240M [00:00<?, ?B/s]

...

s1a-s2-raw-s-hh-20241221t213533-20241221(…):   0%|          | 0.00/240M [00:00<?, ?B/s]

INFO:httpx:HTTP Request: GET https://huggingface.co/api/datasets/Maya4/PT1/tree/main/s1a-s2-raw-s-hh-20241221t213533-20241221t213558-057096-0704c4.zarr?recursive=false&expand=false "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://huggingface.co/api/datasets/Maya4/PT1/tree/main/s1a-s2-raw-s-hh-20241221t213533-20241221t213558-057096-0704c4.zarr%2Faz?recursive=false&expand=false "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD https://huggingface.co/datasets/Maya4/PT1/resolve/main/s1a-s2-raw-s-hh-20241221t213533-20241221t213558-057096-0704c4.zarr/az/c/1/0 "HTTP/1.1 302 Found"


Trying to download file s1a-s2-raw-s-hh-20241221t213533-20241221t213558-057096-0704c4.zarr/az/c/1/0 from repo Maya4/PT1


s1a-s2-raw-s-hh-20241221t213533-20241221(…):   0%|          | 0.00/240M [00:00<?, ?B/s]

Batch 2: X shape: torch.Size([32, 1000, 1000]) | Y shape: torch.Size([32, 1000, 1000])
